In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df = df.dropna()

In [5]:
X = df.drop('label', axis=1)
y = df['label']

In [6]:
X.shape

(18285, 4)

In [7]:
y.shape

(18285,)

In [8]:
import nltk 
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [9]:
vocab_size = 5000

In [10]:
messages = X.copy()

In [11]:
messages.reset_index(inplace=True)

In [12]:
wn = WordNetLemmatizer()
corpus = []
for i in range(0, len(messages)):
    # print(i)
    temp = re.sub('[^a-zA-Z]',' ', messages['title'][i])
    temp = temp.lower()
    temp = temp.split()
    temp = [wn.lemmatize(word) for word in temp if word not in stopwords.words('english')]
    temp = ' '.join(temp)
    corpus.append(temp)

In [13]:
corpus[:6]

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart']

In [14]:
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.layers import LSTM, Dense, Dropout

In [15]:
one_hot_data = [one_hot(words, vocab_size) for words in corpus]

In [16]:
one_hot_data[:3]

[[1992, 421, 590, 2286, 4617, 4248, 3673, 2838, 4207, 339],
 [2171, 3755, 4247, 1643, 3631, 3898, 2901],
 [894, 4124, 320, 4278]]

In [17]:
sent_length = 20
embedded_docs = pad_sequences(one_hot_data, padding='pre', maxlen=sent_length)

In [18]:
embedded_docs[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1992,
         421,  590, 2286, 4617, 4248, 3673, 2838, 4207,  339],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0, 2171, 3755, 4247, 1643, 3631, 3898, 2901],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,  894, 4124,  320, 4278]])

In [37]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 20, 40)            200000    
                                                                 
 dropout_2 (Dropout)         (None, 20, 40)            0         
                                                                 
 lstm_2 (LSTM)               (None, 100)               56400     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [38]:
len(embedded_docs), y.shape

(18285, (18285,))

In [39]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [40]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [41]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=64)

Epoch 1/2
192/192 [==============================] - 7s 23ms/step - loss: 0.3404 - accuracy: 0.8359 - val_loss: 0.1966 - val_accuracy: 0.9157
Epoch 2/2
192/192 [==============================] - 3s 17ms/step - loss: 0.1456 - accuracy: 0.9449 - val_loss: 0.1920 - val_accuracy: 0.9215


In [42]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [43]:
y_pred_temp = model.predict(X_test)
y_pred = np.argmax(y_pred_temp, axis=1)


In [44]:
confusion_matrix(y_test, y_pred)

array([[3419,    0],
       [2616,    0]], dtype=int64)

In [45]:
accuracy_score(y_test, y_pred)

0.5665285832642917